In [16]:
import pandas as pd
import numpy as np

# COVID-19

In [17]:
# Total cases

In [18]:
world_total = pd.read_csv('time_series/total-world-covid19.csv')
world_total['Date_Confirmed'] = pd.DatetimeIndex(world_total['Date_Confirmed'])
world_total = world_total[['Date_Confirmed','US','United Kingdom','Switzerland','Italy','India','Australia','Mexico']]

In [19]:
# Daily cases

In [20]:
world_daily = pd.read_csv('time_series/daily-world_covid19.csv')
world_daily['Date_Confirmed'] = pd.DatetimeIndex(world_daily['Date_Confirmed'])
world_daily = world_daily[['Date_Confirmed','US','United Kingdom','Switzerland','Italy','India','Australia','Mexico']]

In [21]:
world_daily

,Date_Confirmed,US,United Kingdom,Switzerland,Italy,India,Australia,Mexico
0,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-24,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-26,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
94,2020-04-25,32796.0,448.0,217.0,2357.0,1753.0,2.0,970.0
95,2020-04-26,27631.0,406.0,167.0,2324.0,1607.0,2.0,835.0
96,2020-04-27,22412.0,392.0,103.0,1739.0,1561.0,1.0,852.0
97,2020-04-28,24385.0,364.0,100.0,2091.0,1873.0,3.0,1223.0


In [22]:
# Process

In [23]:
total = world_total
total = total.fillna(method='ffill') # First fill missing time series totals with previous row value
#total = total.replace(0, np.nan) # Now convert 0 to NaN

daily = world_daily

In [24]:
# Make sure datetimes are correct

In [25]:
    # Total
total['Date_Confirmed'] = pd.to_datetime(total.Date_Confirmed, format='%Y-%m-%d', errors='coerce')
total = total.set_index(pd.DatetimeIndex(total['Date_Confirmed']))

    # Daily
daily['Date_Confirmed'] = pd.to_datetime(daily.Date_Confirmed, format='%Y-%m-%d', errors='coerce')
daily = daily.set_index(pd.DatetimeIndex(daily['Date_Confirmed']))

In [26]:
# Apply rolling window

In [27]:
total = total.dropna().rolling(window=7).mean().round().reset_index()
daily = daily.dropna().rolling(window=7).mean().round().reset_index()

In [28]:
# Rename index

In [29]:
total = total.rename(columns={'index':'Date_Confirmed'})
daily = daily.rename(columns={'index':'Date_Confirmed'})

In [30]:
# Melt to long format

In [31]:
tbl_total = pd.DataFrame(total.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [32]:
tbl_daily = pd.DataFrame(daily.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [33]:
tbl = pd.merge(tbl_total, tbl_daily, how='inner', on=['Date_Confirmed', 'Region'])

In [34]:
tbl

0_x     0_y
Date_Confirmed Region                          
2020-01-28     US                   3.0     1.0
               United Kingdom       0.0     0.0
               Switzerland          0.0     0.0
               Italy                0.0     0.0
               India                0.0     0.0
...                                 ...     ...
2020-04-29     Switzerland      28995.0   163.0
               Italy           197215.0  2323.0
               India            27945.0  1670.0
               Australia          839.0     2.0
               Mexico           14729.0  1036.0

[651 rows x 2 columns]

In [35]:
tbl = tbl.rename(columns={'0_x': 'total', '0_y': 'daily'})

In [36]:
tbl.head()

total  daily
Date_Confirmed Region                      
2020-01-28     US                3.0    1.0
               United Kingdom    0.0    0.0
               Switzerland       0.0    0.0
               Italy             0.0    0.0
               India             0.0    0.0

In [37]:
# Convert format to fit this plotly express template (long, instead of wide):
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv
tbl_long = tbl.reset_index()
tbl_long.to_csv('output_data/time_series/rolling-time-series.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/time_series/rolling-time-series.csv'

In [38]:
tbl_long

,Date_Confirmed,Region,total,daily
0,2020-01-28,US,3.0,1.0
1,2020-01-28,United Kingdom,0.0,0.0
2,2020-01-28,Switzerland,0.0,0.0
3,2020-01-28,Italy,0.0,0.0
4,2020-01-28,India,0.0,0.0
...,...,...,...,...
646,2020-04-29,Switzerland,28995.0,163.0
647,2020-04-29,Italy,197215.0,2323.0
648,2020-04-29,India,27945.0,1670.0
649,2020-04-29,Australia,839.0,2.0
